In [1]:
print("AIES PROJECT")

AIES PROJECT


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Paths
pothole_dir = 'Path to pothole_pics folder'
no_pothole_dir = 'Path to other_pics folder'
model_path = 'pothole_detection_model.h5'
img_size = 100

# Load and preprocess images
def load_images(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(label)
    return images, labels

# Load dataset
pothole_images, pothole_labels = load_images(pothole_dir, 1)
no_pothole_images, no_pothole_labels = load_images(no_pothole_dir, 0)
images = np.array(pothole_images + no_pothole_images) / 255.0
images = np.expand_dims(images, axis=-1)
labels = np.array(pothole_labels + no_pothole_labels)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Model definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile and train
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Save the trained model
model.save(model_path)

# Reload model for prediction
model = tf.keras.models.load_model(model_path)

# Pothole prediction on a single image
def predict_pothole_on_image(image_path, model):
    original_img = cv2.imread(image_path)
    if original_img is None:
        print("Image not found.")
        return
    
    display_img = original_img.copy()
    img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (img_size, img_size)) / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)[0][0]

    if prediction > 0.5:
        print(f"Pothole detected with confidence: {prediction:.2f}")
        h, w, _ = display_img.shape
        start = (int(w * 0.1), int(h * 0.1))
        end = (int(w * 0.9), int(h * 0.9))
        cv2.rectangle(display_img, start, end, (0, 0, 255), 2)
        cv2.putText(display_img, "Pothole", (start[0], start[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        print(f"No pothole detected with confidence: {1 - prediction:.2f}")

    cv2.imshow("Image Prediction", display_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Pothole detection on video
def predict_pothole_on_video(video_path, model):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (1280, 720))

        display_frame = frame.copy()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (img_size, img_size)) / 255.0
        input_img = np.expand_dims(resized, axis=-1)
        input_img = np.expand_dims(input_img, axis=0)

        prediction = model.predict(input_img)[0][0]

        if prediction > 0.5:
            h, w, _ = display_frame.shape
            start = (int(w * 0.1), int(h * 0.1))
            end = (int(w * 0.9), int(h * 0.9))
            cv2.rectangle(display_frame, start, end, (0, 0, 255), 2)
            cv2.putText(display_frame, "Pothole", (start[0], start[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow("Video Pothole Detection", display_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# ==== Prediction on Webcam ====
def predict_on_webcam(model):
    cap = cv2.VideoCapture(0)  # 0 = default webcam

    if not cap.isOpened():
        print("Error: Could not access webcam.")
        return

    print("Starting webcam... Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        display_frame = frame.copy()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (img_size, img_size)) / 255.0
        input_img = np.expand_dims(gray, axis=-1)
        input_img = np.expand_dims(input_img, axis=0)

        prediction = model.predict(input_img)[0][0]

        if prediction > 0.5:
            h, w, _ = display_frame.shape
            cv2.rectangle(display_frame, (int(w*0.1), int(h*0.1)), (int(w*0.9), int(h*0.9)), (0, 0, 255), 2)
            cv2.putText(display_frame, "Pothole", (int(w*0.1), int(h*0.1)-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow("Pothole Detection - Webcam", display_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# ==== Run image or video or webcam detection here ====
# For image:
image_path = 'Path to test_image2.jpg'
predict_pothole_on_image(image_path, model)

# # For video:
video_path = 'Path to test_video.mp4' 
predict_pothole_on_video(video_path, model)

# For webcam prediction:
predict_on_webcam(model)


D:\New folder\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 22s 22s/step - accuracy: 0.5000 - loss: 0.6900 - val_accuracy: 0.5000 - val_loss: 0.7556
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - accuracy: 0.5625 - loss: 0.5962 - val_accuracy: 0.5000 - val_loss: 1.2723
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - accuracy: 0.5000 - loss: 1.1655 - val_accuracy: 0.7500 - val_loss: 0.6620
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.9375 - loss: 0.4420 - val_accuracy: 0.5000 - val_loss: 1.2828
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.5625 - loss: 0.7408 - val_accuracy: 0.5000 - val_loss: 0.8829
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.7500 - loss: 0.4691 - val_accuracy: 0.5000 - val_loss: 0.6433
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - accuracy: 0.8125 - loss: 0.4289 - val_accuracy: 0.5000 - val_loss: 0.6750
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - accuracy: 0.7500 - loss: 0.4512 - val_accuracy: 0.7500 - val_loss: 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Pothole detected with confidence: 0.97
